In [ ]:
import numpy as np 
import pandas as pd 
import cv2, pickle
from PIL import Image 
import pathlib
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.applications import ResNet50
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

Using TensorFlow backend.


In [ ]:
IMG_WIDTH=256
IMG_HEIGHT=256
BATCH_SIZE = 5

In [ ]:
train = pickle.load(open("/content/drive/My Drive/Colab Notebooks/dl_comptetion/dataset/train.npy",'rb'))
test = pickle.load(open("/content/drive/My Drive/Colab Notebooks/dl_comptetion/dataset/test.npy",'rb'))

In [ ]:
train = np.array(train)
test = np.array(test)

In [ ]:
train_data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/dl_comptetion/dataset/train.csv")
test_data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/dl_comptetion/dataset/test.csv")
encoder = LabelEncoder()
encoder.fit(train_data["target"])
encoded_Y = encoder.transform(train_data["target"])
y_labels = np_utils.to_categorical(encoded_Y)

In [ ]:
train_data.head()

,Image,target
0,96.jpg,manipuri
1,163.jpg,bharatanatyam
2,450.jpg,odissi
3,219.jpg,kathakali
4,455.jpg,odissi


## RESNET

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
import tensorflow.keras as keras

resnet = ResNet50(include_top=False, weights='imagenet', input_shape=(IMG_HEIGHT,IMG_WIDTH,3),pooling='max')

output = resnet.layers[-1].output
output = tf.keras.layers.Flatten()(output)
resnet = Model(resnet.input, output)

res_name = []
for layer in resnet.layers:
    res_name.append(layer.name)

In [ ]:
set_trainable = False
for layer in resnet.layers:
    if layer.name in res_name[-22:]:
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout

model = Sequential()
model.add(resnet)
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_2 (Model)              (None, 2048)              23587712  
_________________________________________________________________
dense_6 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dropout_5 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 8)                 8200      
Total params: 26,743,688
Trainable params: 12,087,304
Non-trainable params: 14,656,384
_________________________________

In [ ]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8,
                                              restore_best_weights=False
                                              )

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                   factor=0.2,
                                   patience=4,
                                   verbose=1,
                                   min_delta=5*1e-3,min_lr = 5*1e-7,
                                   )

In [ ]:
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics=['accuracy',tfa.metrics.F1Score(num_classes=8)])

In [ ]:
p=[]
for i in [10,15,20]:
  model.fit(train,y_labels,
          steps_per_epoch=np.ceil(float(train.shape[0]) / float(BATCH_SIZE)),
          epochs = i,callbacks=[early_stop,reduce_lr])
  pred = model.predict(test)
  pred = np.argmax(pred, axis=1)

  pred = encoder.inverse_transform(pred)
  p.append(pred)

l=[]
for i in range(p[0].shape[0]):
  l.append([p[0][i],p[1][i],p[2][i]])
with open('/content/drive/My Drive/Colab Notebooks/dl_comptetion/dataset/listfile.txt', 'w') as filehandle:
    for listitem in l:
        filehandle.write('%s\n' % listitem)
a=[]
for i in l:
  a.append(max(i,key=i.count))
result = pd.DataFrame(a, test_data["Image"], columns=["target"])
result.to_csv("/content/drive/My Drive/Colab Notebooks/dl_comptetion/dataset/sample.csv")

Epoch 1/10
73/73 [==============================] - 111s 2s/step - loss: 0.0525 - accuracy: 0.9808 - f1_score: 0.9806 - lr: 1.0000e-04
Epoch 2/10
73/73 [==============================] - 111s 2s/step - loss: 0.0460 - accuracy: 0.9918 - f1_score: 0.9920 - lr: 1.0000e-04
Epoch 3/10
73/73 [==============================] - 110s 2s/step - loss: 0.1275 - accuracy: 0.9560 - f1_score: 0.9557 - lr: 1.0000e-04
Epoch 4/10
73/73 [==============================] - 110s 2s/step - loss: 0.1629 - accuracy: 0.9698 - f1_score: 0.9703 - lr: 1.0000e-04
Epoch 5/10
73/73 [==============================] - 107s 1s/step - loss: 0.0919 - accuracy: 0.9780 - f1_score: 0.9784 - lr: 1.0000e-04
Epoch 6/10
73/73 [==============================] - 109s 1s/step - loss: 0.0531 - accuracy: 0.9863 - f1_score: 0.9860 - lr: 1.0000e-04
Epoch 7/10
73/73 [==============================] - 109s 1s/step - loss: 0.0741 - accuracy: 0.9780 - f1_score: 0.9767 - lr: 1.0000e-04
Epoch 8/10
73/73 [==============================] - 109

In [ ]:
model.fit(train,y_labels,
          steps_per_epoch=np.ceil(float(train.shape[0]) / float(BATCH_SIZE)),
          epochs = 100,callbacks=[early_stop,reduce_lr])
pred = model.predict(test)
pred = np.argmax(pred, axis=1)

pred = encoder.inverse_transform(pred)
result = pd.DataFrame(pred, test_data["Image"], columns=["target"])
result.to_csv("/content/drive/My Drive/Colab Notebooks/dl_comptetion/dataset/sample.csv")

Epoch 1/100
73/73 [==============================] - 112s 2s/step - loss: 0.0293 - accuracy: 0.9918 - f1_score: 0.9919 - lr: 1.0000e-04
Epoch 2/100
73/73 [==============================] - 110s 2s/step - loss: 0.0297 - accuracy: 0.9863 - f1_score: 0.9860 - lr: 1.0000e-04
Epoch 3/100
73/73 [==============================] - 114s 2s/step - loss: 0.0947 - accuracy: 0.9753 - f1_score: 0.9751 - lr: 1.0000e-04
Epoch 4/100
73/73 [==============================] - 111s 2s/step - loss: 0.0527 - accuracy: 0.9918 - f1_score: 0.9920 - lr: 1.0000e-04
Epoch 5/100
73/73 [==============================] - 110s 2s/step - loss: 0.0203 - accuracy: 0.9945 - f1_score: 0.9942 - lr: 1.0000e-04
Epoch 6/100
73/73 [==============================] - 111s 2s/step - loss: 0.0027 - accuracy: 0.9973 - f1_score: 0.9974 - lr: 1.0000e-04
Epoch 7/100
73/73 [==============================] - 108s 1s/step - loss: 1.3697e-04 - accuracy: 1.0000 - f1_score: 1.0000 - lr: 1.0000e-04
Epoch 8/100
73/73 [=========================